<a href="https://colab.research.google.com/github/Walidsati/AAI_612O/blob/main/Notebook6.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# AAI612: Deep Learning & its Applications

*Notebook 6.4: How do you feel about that movie?*


In [2]:
import pandas as pd
import numpy as np
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

df = pd.DataFrame()
df = pd.read_csv('https://raw.githubusercontent.com/harmanani/AAI612/main/Week6/data/movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1


In [3]:
X_train = df.loc[:24999, 'review'].values
y_train = df.loc[:24999, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [4]:
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

In [5]:
# summarize size
print("Training data: ")
print(X.shape)
print(y.shape)

Training data: 
(50000,)
(50000,)


In [6]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



tokenizer_obj = Tokenizer()
total_reviews = X_train + X_test
tokenizer_obj.fit_on_texts(total_reviews)

# pad sequences
max_length = 100  # try other options like mean

# define vocabulary size
vocab_size = len(tokenizer_obj.word_index) + 1

X_train_tokens = tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')

In [7]:
print(vocab_size)

125602


In [8]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from tensorflow.keras.layers import Embedding

EMBEDDING_DIM = 100

print('Build model...')

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_shape=(max_length,)))
model.add(GRU(units=32,  dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print('Summary of the built model...')
print(model.summary())

Build model...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Summary of the built model...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 100)            │      12,560,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 32)                  │          12,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,573,097 (47.96 MB)

 Trainable params: 12,573,097 (47.96 MB)

 Non-trainable params: 0 (0.00 B)

None


In [9]:
print('Train...')

model.fit(X_train_pad, y_train, batch_size=128, epochs=25, validation_data=(X_test_pad, y_test), verbose=2)

Train...
Epoch 1/25
196/196 - 56s - 284ms/step - accuracy: 0.7067 - loss: 0.5565 - val_accuracy: 0.8082 - val_loss: 0.4277
Epoch 2/25
196/196 - 49s - 250ms/step - accuracy: 0.8561 - loss: 0.3537 - val_accuracy: 0.8126 - val_loss: 0.4214
Epoch 3/25
196/196 - 48s - 245ms/step - accuracy: 0.9104 - loss: 0.2448 - val_accuracy: 0.8323 - val_loss: 0.4019
Epoch 4/25
196/196 - 48s - 246ms/step - accuracy: 0.9449 - loss: 0.1622 - val_accuracy: 0.8264 - val_loss: 0.4462
Epoch 5/25
196/196 - 49s - 248ms/step - accuracy: 0.9636 - loss: 0.1090 - val_accuracy: 0.8214 - val_loss: 0.5132
Epoch 6/25
196/196 - 48s - 246ms/step - accuracy: 0.9755 - loss: 0.0724 - val_accuracy: 0.8163 - val_loss: 0.5613
Epoch 7/25
196/196 - 49s - 248ms/step - accuracy: 0.9860 - loss: 0.0458 - val_accuracy: 0.8136 - val_loss: 0.6375
Epoch 8/25
196/196 - 49s - 248ms/step - accuracy: 0.9916 - loss: 0.0294 - val_accuracy: 0.8052 - val_loss: 0.7248
Epoch 9/25
196/196 - 49s - 249ms/step - accuracy: 0.9944 - loss: 0.0198 - val_a

In [10]:
print('Testing...')
score, acc = model.evaluate(X_test_pad, y_test, batch_size=128)

print('Test score:', score)
print('Test accuracy:', acc)

print("Accuracy: {0:.2%}".format(acc))

Testing...
196/196 ━━━━━━━━━━━━━━━━━━━━ 10s 48ms/step - accuracy: 0.7350 - loss: 1.8081
Test score: 1.4687045812606812
Test accuracy: 0.7805200219154358
Accuracy: 78.05%


In [11]:
#Let us test some  samples
test_sample_1 = "This movie is fantastic! I really like it because it is so good!"
test_sample_2 = "Good movie!"
test_sample_3 = "Maybe I like this movie."
test_sample_4 = "Not to my taste, will skip and watch another movie"
test_sample_5 = "if you like action, then this movie might be good for you."
test_sample_6 = "Bad movie!"
test_sample_7 = "Not a good movie!"
test_sample_8 = "This movie really sucks! Can I get my money back please?"
test_samples = [test_sample_1, test_sample_2, test_sample_3, test_sample_4, test_sample_5, test_sample_6, test_sample_7, test_sample_8]

test_samples_tokens = tokenizer_obj.texts_to_sequences(test_samples)
test_samples_tokens_pad = pad_sequences(test_samples_tokens, maxlen=max_length)

#predict
model.predict(x=test_samples_tokens_pad)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 443ms/step


array([[9.9917346e-01],
       [8.6046821e-01],
       [6.4594942e-01],
       [8.2323625e-04],
       [7.7668315e-01],
       [1.6087909e-01],
       [6.9277000e-01],
       [2.6972051e-05]], dtype=float32)

In [15]:
#let us check how the model predicts
classes = model.predict(X_test_pad[:10], batch_size=128)
for i in range (0,10):
    if(classes[i] > 0.5 and y_test[i] == 1 or (classes[i] <= 0.5 and y_test[i] == 0)):
        print( classes[i], y_test[i], " Right prediction")
    else :
        print( classes[i], y_test[i], " Wrong prediction")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step
[0.5109366] 0  Wrong prediction
[0.51292324] 1  Right prediction
[0.51467] 1  Right prediction
[0.50581425] 0  Wrong prediction
[0.49564552] 1  Wrong prediction
[0.50975597] 1  Right prediction
[0.50266105] 1  Right prediction
[0.49992383] 0  Right prediction
[0.505367] 0  Wrong prediction
[0.50748336] 1  Right prediction


In [16]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import Dense, Embedding, LSTM, GRU
from tensorflow.keras.layers import Embedding

# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

print('Build model...')

model = Sequential()
model.add(Embedding(top_words, 100, input_shape=(max_words,)))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Build model...


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 500, 100)            │         500,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          17,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 517,057 (1.97 MB)

 Trainable params: 517,057 (1.97 MB)

 Non-trainable params: 0 (0.00 B)

None


In [17]:
print('Train...')

model.fit(X_train, y_train, batch_size=128, epochs=12, validation_data=(X_test, y_test), verbose=2)

Train...
Epoch 1/12
196/196 - 282s - 1s/step - accuracy: 0.7553 - loss: 0.5052 - val_accuracy: 0.8351 - val_loss: 0.3862
Epoch 2/12
196/196 - 276s - 1s/step - accuracy: 0.8286 - loss: 0.3936 - val_accuracy: 0.8087 - val_loss: 0.4247
Epoch 3/12
196/196 - 276s - 1s/step - accuracy: 0.8594 - loss: 0.3411 - val_accuracy: 0.8464 - val_loss: 0.3630
Epoch 4/12
196/196 - 276s - 1s/step - accuracy: 0.8603 - loss: 0.3442 - val_accuracy: 0.8408 - val_loss: 0.3745
Epoch 5/12
196/196 - 277s - 1s/step - accuracy: 0.8738 - loss: 0.3101 - val_accuracy: 0.8438 - val_loss: 0.3642
Epoch 6/12
196/196 - 276s - 1s/step - accuracy: 0.8886 - loss: 0.2836 - val_accuracy: 0.7958 - val_loss: 0.4714
Epoch 7/12
196/196 - 275s - 1s/step - accuracy: 0.8848 - loss: 0.2861 - val_accuracy: 0.8567 - val_loss: 0.3548
Epoch 8/12
196/196 - 277s - 1s/step - accuracy: 0.8939 - loss: 0.2627 - val_accuracy: 0.8538 - val_loss: 0.3611
Epoch 9/12
196/196 - 277s - 1s/step - accuracy: 0.9052 - loss: 0.2433 - val_accuracy: 0.8548 - 

In [18]:
score, acc = model.evaluate(X_test, y_test, batch_size=128)

print('Test score:', score)
print('Test accuracy:', acc)
print("Accuracy: %.2f%%" % (acc*100))

196/196 ━━━━━━━━━━━━━━━━━━━━ 49s 250ms/step - accuracy: 0.8600 - loss: 0.3752
Test score: 0.3676946759223938
Test accuracy: 0.8619999885559082
Accuracy: 86.20%


The time to train a GRU is less than LSTM network.